In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [ ]:
%matplotlib tk

In [ ]:
# collect image names
BASE_PATH = '/home/daniel/Schreibtisch/bachelorarbeit/src'
TEST_PATH = '{}/test_images/images/std_raw/'.format(BASE_PATH)

img_names = []
folders = os.listdir(TEST_PATH)
for f in ['25']:
    path = '{}/{}'.format(TEST_PATH, f)
    if not os.path.isdir(path):
        continue
    images = map(lambda x: '{}/{}'.format(f, x), os.listdir(path))
    img_names += list(images)

In [ ]:
code = '191136'
for i, name in enumerate(img_names[:]):
    if code in name:
        print(i, name)

In [ ]:
# load image
i = 6
img = cv2.imread(TEST_PATH + img_names[i], 1)

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# normalize
img_norm = img - np.min(img)
img_norm = (img_norm.astype('uint64') * 255 // np.max(img_norm)).astype('uint8')

plt.imshow(cv2.cvtColor(img_norm, cv2.COLOR_BGR2RGB))
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# development
t_img = cv2.GaussianBlur(img_norm, (21, 21), 5)

plt.imshow(cv2.cvtColor(t_img, cv2.COLOR_BGR2RGB))
plt.xticks([])
plt.yticks([])
plt.show()

hsv = cv2.cvtColor(t_img, cv2.COLOR_BGR2HSV)

low1 = (0, 0, 85)
high1 = (80, 110, 255)

low2 = (120, 0, 110)
high2 = (179, 100, 255)

mask1 = cv2.inRange(hsv, low1, high1)
mask2 = cv2.inRange(hsv, low2, high2)

mask = cv2.bitwise_or(mask1, mask2)

In [ ]:
plt.imshow(hsv)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.imshow(mask, cmap='gray')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
blowout = True
if blowout:
    mask = cv2.dilate(mask, np.ones((15,15), np.uint8), iterations = 1)
contours, _ = cv2.findContours(mask.copy(),
                               cv2.RETR_EXTERNAL,
                               cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
# Find object with the biggest bounding box
mx = (0, 0, 0, 0)      # biggest bounding box so far
mx_area = 0
mx_contourIdx = 0
org_area = img.shape[0] * img.shape[1]  # area if original image

for i, cont in enumerate(contours):
    x, y, w, h = cv2.boundingRect(cont)
    area = w * h
    if area > mx_area and area < org_area:
        mx = x, y, w, h
        mx_area = area
        mx_contourIdx = i

In [ ]:
# blur outside of contour
blur = True
if blur:
    blurred_image = cv2.GaussianBlur(img_norm,(31, 31), 10)
    
    mask = np.zeros(img_norm.shape, dtype=np.uint8)
    cv2.fillPoly(mask, [contours[mx_contourIdx]], (255, 255, 255))
    mask_inverse = np.ones(mask.shape).astype(np.uint8) * 255 - mask
    
    img_black = cv2.bitwise_and(img_norm, mask) + cv2.bitwise_and(blurred_image, mask_inverse)
    
    plt.imshow(cv2.cvtColor(img_black, cv2.COLOR_BGR2RGB))
    plt.xticks([])
    plt.yticks([])
    plt.show()
else:
    img_black = img_norm

In [ ]:
# make bounding box quadratic
x, y, w, h = mx

if w < h:
    x -= (h - w) // 2
    w = h
elif h < w:
    y -= (w - h) // 2
    h = w
    
print(x, y, w, h)

In [ ]:
border = 0.
if border > 0.:
    # add border to bounding box
    border = int(w * border)

    x -= border
    y -= border
    w += border * 2
    h += border * 2

print(x, y, w, h)

In [ ]:
assert x > 0
assert y > 0
assert x + w < img.shape[1]
assert y + h < img.shape[0]

In [ ]:
# draw contour and box
img_cont = img_black.copy()

img_cont = cv2.drawContours(img_cont,
                            contours,
                            mx_contourIdx,
                            (0, 255, 255), 5)
img_cont = cv2.rectangle(img_cont, (x, y),
                         (x+w, y+h),
                         (255, 0, 255), 5)

plt.imshow(cv2.cvtColor(img_cont, cv2.COLOR_BGR2RGB))
plt.xticks([])
plt.yticks([])
# cv2.imwrite('./test.png', img_cont)
plt.show()

In [ ]:
# crop bounding box from image
roi = img_black[y:y+h, x:x+w]
print(roi.shape)

plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# resize image
dim = (128, 128) # size of rendered images
img_res = cv2.resize(roi, dim)

print('Original Dimensions : ', roi.shape)
print('New Dimensions : ', img_res.shape)

plt.imshow(cv2.cvtColor(img_res, cv2.COLOR_BGR2RGB))
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# normalize
img_norm2 = img_res - np.min(img_res)
img_norm2 = (img_norm2.astype('uint64') * 255 // np.max(img_norm2)).astype('uint8')

plt.imshow(cv2.cvtColor(img_norm2, cv2.COLOR_BGR2RGB))
plt.xticks([])
plt.yticks([])
# cv2.imwrite('./test.png', img_norm2)
plt.show()